In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from decision_vars import DecisionVarSet

In [6]:
# Test decision vars
from decision_vars import DecisionVarSet
dvs = DecisionVarSet(attr_names = ['lb'], name = 'sneaker/')
dvs.add_vars({'x':[1.0, 2.0]}, lb={'x':[0, 0]})
print(dvs)
dvs2 = DecisionVarSet(attr_names = ['lb'], name = 'ouble/')
dvs2.add_vars(init={'x':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
d = {'test':1}
d.update(dvs2)
print(d)

***** Decision Vars *****
Attributes: ['lb']
Vars: 
  sneaker/x: [x_0, x_1], shape: (2, 1)

[0, 0, -inf]
{'sneaker/x': DM([0.1, 0.3]), 'ouble/x': DM(0.5)}
0.5
None
{'test': 1, 'ouble/x': SX(x)}


In [3]:
from robot import *
rob = Robot('config/franka.urdf')

Building robot model from config/franka.urdf with TCP fr3_link8


In [4]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [0, 10, 10],
        'rest':[0, 0, 0.0]}
c = Contact('friendo', pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
print(c.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 'R':ca.DM.ones(3)}))

c2 = Contact('c2',pars, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
print(c2.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 
                    'R':ca.DM.ones(3), 
                    'c2/stiff':ca.DM([10, 0, 0])}))

[0, -3, 5]
[-0, -0, 0]


In [8]:
# Test convergence
from robot import Robot
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, [], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])
rob.build_step(0.1)
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = {'q':q, 'dq':dq, 'tau_input':tau}
state['M_inv'] = rob.inv_mass_fn(state['q'])
#state['c1/stiff'] = ca.DM.ones(3)
%timeit rob.step(**state)
for _ in range(100):
    res = rob.step(**state)
    state['q'] = res['q']
    state['dq'] = res['dq']
res_dict = rob.get_ext_state(state)
print(res_dict['p'])
#print(res_dict['c1/F'])
#print(res_dict['c2/F'])

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c1/', 'c2/']
29.7 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
[0.311901, 0.497578, 0.200615]


In [9]:
from robot import Robot
from contact import Contact
from decision_vars import DecisionVarSet
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])
#print(rob.get_input(H=5))
#print(rob.get_state(H=5))

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c1/', 'c2/']


In [11]:
# Check that the spawner works
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = [])
for name, model in models.items():
    model.build_step(step_size=0.1)
    print(model.step)
    n_in = model.step.size_in(0)
    res = model.step(np.zeros(7), np.zeros(7), np.eye(7), np.zeros(7))
    print(f"{name} has step fn of {model.step}")
    print(f"  which evals to {res}")


Building robot model from config/franka.urdf with TCP fr3_link8
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/', 'contact_2/']
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
free has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([0, 0, 0, 0, 0, 0, 0]), DM([0, 0, 0, 0, 0, 0, 0]), DM(0))
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
plane has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([0, 0.249603, 0, -0.0156002, 0, -0.249603, 0]), DM([0, 2.49603, 0, -0.156002, 0, -2.49603, 0]), DM(0))
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
hinge has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([-0.61072, 0.24960